In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [2]:
#load the trained model, scaler pickle file, onehot

model = load_model('model.h5')

# load encoder and scaler

with  open('onehot_encoder_geo.pkl', 'rb') as file:
    onehot_encoder_geo=pickle.load(file)

with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler = pickle.load(file)

In [3]:
input_data = {
    'CreditScore': 6,
    'Geography': 'Germany',
    'Gender': 'Male',
    'Age': 4,
    'Tenure': 3,
    'Balance': 6000,
    'NumOfProducts': 1,
    'HasCrCard': 0,
    'IsActiveMember': 1,
    'EstimatedSalary': 5
}

In [4]:
df1 = pd.DataFrame(data=[input_data.values()],columns=input_data.keys())
df1

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,6,Germany,Male,4,3,6000,1,0,1,5


In [5]:
df1['Gender'] = label_encoder_gender.transform([input_data['Gender']])[0]

In [6]:
df1

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,6,Germany,1,4,3,6000,1,0,1,5


In [7]:
encoded_geo = onehot_encoder_geo.transform([[input_data['Geography']]]).toarray()
encoded_geo_col_names = onehot_encoder_geo.get_feature_names_out()
encoded_geo_col_names

c:\Users\fifty\anaconda3\envs\churn_project\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [8]:
encoded_geo_df = pd.DataFrame(data=encoded_geo,columns=encoded_geo_col_names)
encoded_geo_df

,Geography_France,Geography_Germany,Geography_Spain
0,0.0,1.0,0.0


In [9]:
df = pd.concat([df1,encoded_geo_df],axis=1)
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,6,Germany,1,4,3,6000,1,0,1,5,0.0,1.0,0.0


In [10]:
df.drop(columns=['Geography'],inplace=True)

In [11]:
df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,6,1,4,3,6000,1,0,1,5,0.0,1.0,0.0


In [12]:
df = scaler.transform(df)

In [13]:
df

array([[-6.69136807,  0.90384899, -3.31666961, -0.70206791, -1.1253033 ,
        -0.9104867 , -1.54916591,  0.97311137, -1.73501147, -0.99975003,
         1.71490137, -0.57176909]])

In [14]:
model.predict(df)

1/1 [==============================] - 0s 147ms/step


array([[0.05032017]], dtype=float32)